In [1]:
import pandas as pd
import numpy as np

In [2]:
click_columns = ['user_id','item_id','time']
train_click_T = pd.DataFrame()
test_click_T = pd.DataFrame()
test_qtime_T = pd.DataFrame()


# 第t个阶段，读取所有文件并合并
t = 4
for i in range(t):
    temp_train_click = pd.read_csv('../data/underexpose_train/underexpose_train_click-%s.csv'%(str(i)),sep=',',names = click_columns )
    temp_test_click = pd.read_csv('../data/underexpose_test/underexpose_test_click-%s.csv'%(str(i)),sep=',',names = click_columns)
    temp_test_qtime = pd.read_csv('../data/underexpose_test/underexpose_test_qtime-%s.csv'%(str(i)),sep=',',names = ['user_id','time'])
    train_click_T = train_click_T.append(temp_train_click)
    test_click_T = test_click_T.append(temp_test_click)
    test_qtime_T = test_qtime_T.append(temp_test_qtime)

# 拼接所有历史点击数据
data_click_T = pd.concat([train_click_T,test_click_T],axis = 0).reset_index(drop = True)


# 计算所有商品的流行度-历史点击次数排序
data_click_T_item = data_click_T.groupby('item_id',as_index= False)['user_id'].agg({'user_dis':'unique','click_cnt':'count'})
data_click_T_item_rank = data_click_T_item.sort_values(by = 'click_cnt',axis = 0,ascending = False)['item_id'].tolist()

In [3]:
whole_dataset = pd.concat([train_click_T,test_click_T]).reset_index(drop=True)

In [4]:
whole_dataset.head(10)

,user_id,item_id,time
0,4965,18,0.983763
1,20192,34,0.983772
2,30128,91,0.983780
3,29473,189,0.983930
4,10625,225,0.983925
5,2426,233,0.983937
6,4043,253,0.983954
7,19451,258,0.983895
8,7944,264,0.983852
9,14329,264,0.983939


In [5]:
final_data = pd.DataFrame()

import tqdm

for user_id in tqdm.tqdm(whole_dataset['user_id'].unique()):
    try:
        user_order = whole_dataset[whole_dataset['user_id'] == user_id].sort_values('time',ascending=True)
        user_order['pre1'] = user_order['item_id'].shift(1)
        user_order['pre2'] = user_order['item_id'].shift(2)
        user_order = user_order[['user_id','pre2','pre1','item_id']].iloc[2:,:].astype('int')
        final_data = final_data.append(user_order)
    except:
        continue

100%|██████████| 26046/26046 [05:43<00:00, 75.80it/s]


In [7]:
final_data = final_data.reset_index(drop=True)
final_data

,user_id,pre2,pre1,item_id
0,4965,33354,43218,18
1,4965,43218,18,66446
2,4965,18,66446,3598
3,4965,66446,3598,55956
4,4965,3598,55956,93772
...,...,...,...,...
1032098,25116,48858,104429,74432
1032099,31056,6082,8772,14576
1032100,31056,8772,14576,110049
1032101,663,9210,95210,16835


In [8]:
final_data.to_csv('semi-data-for-training.csv',index=False)